# localBizdat 
### get data about local businesses (i.e. hotels, restaurants) surrounding the brewery

In [1]:
#import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
Base = automap_base()
import requests
from config import gkey
import json
import numpy as np

In [2]:
#create engine and import data from sqlite database
engine = create_engine('sqlite:///../static/data/bredat.sqlite', echo=False)
Base.metadata.create_all(engine)

In [3]:
Base.prepare(engine, reflect=True)

In [4]:
session = Session(engine)

In [5]:
#create dataframe
brew_df = pd.read_sql_query("SELECT id, latitude, longitude FROM breweries", con=engine)
brew_df

,id,latitude,longitude
0,5016,36.0620103554184,-79.8436043406271
1,5033,36.1327196,-81.677229
2,5035,35.4368388,-82.530580304217
3,5067,35.9915653,-78.90481567784
4,5090,34.9939752576824,-78.9220969691311
...,...,...,...
261,5273,35.59410425,-82.5553147520814
262,5283,35.3206001746298,-82.4578969903109
263,5293,35.0889040369639,-84.0352081706441
264,5304,35.9774896,-78.5100617


In [6]:
#set base url for api call
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [7]:
#create columns for new data from api call
brew_df['hotel_name']=''
brew_df['hotel_address']=''
brew_df['r1_name']=''
brew_df['r1_address']=''
brew_df['r2_name']=''
brew_df['r2_address']=''
brew_df['r3_name']=''
brew_df['r3_address']=''

In [8]:
#hit google places api for hotels

for index, row in brew_df.iterrows():
    loc = f"{str(row.latitude)},{str(row.longitude)}"
    params = {
        "location": loc,
        "type": "lodging",
        "rankby": "distance",
        "key": gkey,
    }
    
    response = requests.get(base_url, params=params)
    results = response.json()["results"]
    
    try:
        brew_df.at[index, 'hotel_name'] = results[0]["name"]
        brew_df.at[index, 'hotel_address'] = results[0]["vicinity"]

    except:
        pass

In [9]:
#hit google places api for restaurants

for index, row in brew_df.iterrows():
    loc = f"{str(row.latitude)},{str(row.longitude)}"
    params = {
        "location": loc,
        "type": "restaurant",
        "rankby": "distance",
        "key": gkey,
    }
    
    response = requests.get(base_url, params=params)
    results = response.json()["results"]
    
    try:
        brew_df.at[index, 'r1_name'] = results[0]["name"]
        brew_df.at[index, 'r1_address'] = results[0]["vicinity"]
        brew_df.at[index, 'r2_name'] = results[1]["name"]
        brew_df.at[index, 'r2_address'] = results[1]["vicinity"]
        brew_df.at[index, 'r3_name'] = results[2]["name"]
        brew_df.at[index, 'r3_address'] = results[2]["vicinity"]

    except:
        pass

In [10]:
brew_df

,id,latitude,longitude,hotel_name,hotel_address,r1_name,r1_address,r2_name,r2_address,r3_name,r3_address
0,5016,36.0620103554184,-79.8436043406271,The Dailey Renewal Retreat Bed and Breakfast,"808 Northridge Street, Greensboro",Fat Dog's Grille & Pub,"3021 Spring Garden Street, Greensboro",Subway,"3011 Spring Garden Street, Greensboro",Kiosco Mexican Grill,"3011 Spring Garden Street, Greensboro"
1,5033,36.1327196,-81.677229,Blowing Rock Brewing Company,"152 Sunset Drive, Blowing Rock",Blowing Rock Brewing Company,"152 Sunset Drive, Blowing Rock",Savannah's Oyster House,"155 Sunset Drive, Blowing Rock",The Village Cafe,"146 Greenway Court, Blowing Rock"
2,5035,35.4368388,-82.530580304217,Fairfield Inn by Marriott Asheville Airport,"155 Underwood Road, Fletcher",Econo Lodge Asheville Airport,"196 Underwood Road, Fletcher",Ruby Tuesdays Airport Rd. Arden N.C.,"308 Overlook Road, Asheville",Ruby Tuesday,"260 Underwood Road, Fletcher"
3,5067,35.9915653,-78.90481567784,Aloft Durham Downtown,"345 Blackwell Street, Durham",Saladelia @ American Tobacco Campus,"406 Blackwell Street, Durham",Boricua Soul at American Tobacco Campus,"406 Blackwell Street #150, Durham",Mellow Mushroom,"410 Blackwell Street, Durham"
4,5090,34.9939752576824,-78.9220969691311,Town & Country Estates,"3931 Legion Road, Hope Mills",Eagle Snacks,"3641 Legion Road, Hope Mills",Bruce & Mickey's,"3154 Legion Road, Hope Mills",McDonald's,"3075 Legion Road, Fayetteville"
...,...,...,...,...,...,...,...,...,...,...,...
261,5273,35.59410425,-82.5553147520814,Downtown Inn & Suites,"120 Patton Avenue, Asheville",Addissae Ethiopian Restaurant,"48 Commerce Street, Asheville",Kathmandu Kitchen,"90 Patton Avenue, Asheville",Red Ginger Dimsum & Tapas,"82 Patton Avenue Suite 100, Asheville"
262,5283,35.3206001746298,-82.4578969903109,The Charleston Inn,"755 North Main Street, Hendersonville",Baires Food Truck,"327-317 7th Avenue East, Hendersonville",Marco's Pizza,"310 7th Avenue East, Hendersonville",Daddy D's Suber Soulfood,"411 7th Avenue East, Hendersonville"
263,5293,35.0889040369639,-84.0352081706441,Bear Hut Cabin,"430 Butterball Blvd, Murphy",Valley River Brewery,"71 Tennessee Street, Murphy",Red Brick Deli,"89 Tennessee Street, Murphy",Bistro 29,"29 Tennessee Street #2938, Murphy"
264,5304,35.9774896,-78.5100617,Wake Forest Masonic Lodge #282,"220 Wait Avenue, Wake Forest",Shorty's Famous Hot Dogs,"214 South White Street, Wake Forest",Sugar Magnolia Cafe,"219 South White Street, Wake Forest",Pizza Amore,"238 South White Street, Wake Forest"


In [11]:
# add data to sqlite database
brew_df[['id','hotel_name','hotel_address','r1_name','r1_address','r2_name','r2_address','r3_name','r3_address']].to_sql('localbusinesses', con=engine, if_exists='replace')

In [12]:
#test sqlite data
engine.execute("SELECT * FROM localbusinesses").fetchall()[0]

(0, 5016, 'The Dailey Renewal Retreat Bed and Breakfast', '808 Northridge Street, Greensboro', "Fat Dog's Grille & Pub", '3021 Spring Garden Street, Greensboro', 'Subway', '3011 Spring Garden Street, Greensboro', 'Kiosco Mexican Grill', '3011 Spring Garden Street, Greensboro')